In [1]:
%pip install llama-index


  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.
instructor 1.7.2 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.
langchain-google-vertexai 2.0.13 requires httpx<0.29.0,>=0.28.0, but you have httpx 0.27.2 which is incompatible.
langchain-pinecone 0.2.2 requires aiohttp<3.11,>=3.10, but you have aiohttp 3.11.11 which is incompatible.
llama-index-llms-huggingface 0.4.1 requires huggingface-hub<0.24.0,>=0.23.0, but you have huggingface-hub 0.28.1 which is incompatible.
open-webui 0.5.12 requires chromadb==0.6.2, but you have chromadb 0.5.23 which is incompatible.
open-webui 0.5.12 requires google-generativeai==0.7.2, but you have google-generativeai 0.8.4 which is incompatible.
open-webui 0.5.12 requires langchain==0.3.7, 

In [1]:
from dotenv import load_dotenv

load_dotenv()
import os 

api_key=os.getenv("GITHUB_API_KEY")
tavily_api_key=os.getenv("TAVILY_API_KEY")




In [2]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", api_key=api_key,api_base="https://models.inference.ai.azure.com")

In [3]:
resp = llm.complete("How did india win the world cup 2024?")

In [5]:
resp.text

"As of my last knowledge update in October 2023, the 2024 ICC Men's T20 World Cup had not yet taken place, and I do not have information about its outcome. To find out how India won the 2024 World Cup, I recommend checking the latest sports news or official cricket websites for the most current information."

In [ ]:
%pip install tavily-python

In [6]:
from tavily import AsyncTavilyClient


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

In [7]:
from llama_index.core.agent.workflow import  AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

In [8]:
response = await workflow.run(user_msg="How did india win the world cup 2024?")
print(str(response))

India won the ICC T20 World Cup 2024 by defeating South Africa in a thrilling final held at the Kensington Oval in Bridgetown, Barbados, on June 29, 2024. India emerged victorious by just 7 runs, marking a significant achievement as they became the first team to win the T20 World Cup without losing a single match throughout the tournament.

In the final, India showcased a strong performance, building on their earlier successes in the tournament, including a decisive 68-run victory over England in the semifinals. The win ended a 13-year wait for an ICC Cricket World Cup trophy for India.

For more details, you can read the full articles on [Sporting News](https://www.sportingnews.com/in/cricket/news/how-india-won-2024-t20-world-cup-2024/fba73d5f370de244e73a5289) and [Al Jazeera](https://www.aljazeera.com/sports/2024/6/29/india-vs-south-africa-icc-t20-world-cup-final-result-match-report-champions-rohit-markram-bumrah-shamsi).


In [9]:
from llama_index.core.workflow import Context

ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Logan, nice to meet you!", ctx=ctx
)
print(str(response))

Nice to meet you, Logan! How can I assist you today?


In [10]:
response = await workflow.run(user_msg="What is my name?", ctx=ctx)
print(str(response))

Your name is Logan.


In [11]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [12]:
response = await workflow.run(
    user_msg="Do you still remember my name?", ctx=restored_ctx
)
print(str(response))

Yes, I remember your name is Logan.


In [13]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon?")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
        # print(event.response)  # the current full response
        # print(event.raw)  # the raw llm api response
        # print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentInput):
    #    print(event.input)  # the current input messages
    #    print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentOutput):
    #    print(event.response)  # the current full response
    #    print(event.tool_calls)  # the selected tool calls, if any
    #    print(event.raw)  # the raw llm api response
    # elif isinstance(event, ToolCallResult):
    #    print(event.tool_name)  # the tool name
    #    print(event.tool_kwargs)  # the tool kwargs
    #    print(event.tool_output)  # the tool output
    # elif isinstance(event, ToolCall):
    #     print(event.tool_name)  # the tool name
    #     print(event.tool_kwargs)  # the tool kwargs

The current weather in Saskatoon is as follows:

- **Temperature**: -3.7°C (25.3°F)
- **Condition**: Mist
- **Wind**: 6.7 mph (10.8 kph) from the South
- **Humidity**: 93%
- **Visibility**: 2.4 km (1.0 miles)
- **Feels Like**: -8.0°C (17.7°F)

For more details, you can check the full report [here](https://www.weatherapi.com/).

In [ ]:
from llama_index.core.workflow import Context


async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"


workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

response = await workflow.run(user_msg="My name is Logan", ctx=ctx)
print(str(response))

state = await ctx.get("state")
print(state["name"])